### Let's construct LeNet in Keras!

#### First let's load and prep our MNIST data

In [1]:
# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.datasets import mnist
from keras.utils import np_utils
import keras

# loads the MNIST dataset
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

# Lets store the number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

# Getting our date in the right 'shape' needed for Keras
# We need to add a 4th dimenion to our date thereby changing our
# Our original image shape of (60000,28,28) to (60000,28,28,1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# store the shape of a single image 
input_shape = (img_rows, img_cols, 1)

# change our image type to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255
x_test /= 255

# Now we one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

### Now let's create our layers to replicate LeNet

In [2]:
# create model
model = Sequential()

# 2 sets of CRP (Convolution, RELU, Pooling)
model.add(Conv2D(20, (5, 5),
                 padding = "same", 
                 input_shape = input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2) ))

model.add(Conv2D(50, (5, 5),
                 padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

# Fully connected layers (w/ RELU)
model.add(Flatten())
          
# FC NN
model.add(Dense(500))
model.add(Activation("relu"))

# Softmax (for classification)
model.add(Dense(num_classes))
model.add(Activation("softmax"))
           
model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adadelta(),
              metrics = ['accuracy'])
    
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 20)        520       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 20)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 50)        25050     
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 50)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2450)              0

### Now let us train LeNet on our MNIST Dataset

In [3]:
# Training Parameters
batch_size = 128
epochs = 10

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

model.save("mnist_LeNet.h5")


Epoch 1/10
469/469 [==============================] - 112s 135ms/step - loss: 2.2911 - accuracy: 0.1174 - val_loss: 2.2434 - val_accuracy: 0.1988
Epoch 2/10
469/469 [==============================] - 61s 130ms/step - loss: 2.2283 - accuracy: 0.2536 - val_loss: 2.1746 - val_accuracy: 0.4248
Epoch 3/10
469/469 [==============================] - 58s 124ms/step - loss: 2.1564 - accuracy: 0.4669 - val_loss: 2.0864 - val_accuracy: 0.5996
Epoch 4/10
469/469 [==============================] - 60s 128ms/step - loss: 2.0643 - accuracy: 0.6045 - val_loss: 1.9673 - val_accuracy: 0.6789
Epoch 5/10
469/469 [==============================] - 58s 124ms/step - loss: 1.9384 - accuracy: 0.6766 - val_loss: 1.8047 - val_accuracy: 0.7227
Epoch 6/10
469/469 [==============================] - 57s 122ms/step - loss: 1.7660 - accuracy: 0.7169 - val_loss: 1.5974 - val_accuracy: 0.7505
Epoch 7/10
469/469 [==============================] - 60s 128ms/step - loss: 1.5541 - accuracy: 0.7402 - val_loss: 1.3642 - val_a

In [10]:
# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 3s 8ms/step - loss: 1.0413 - accuracy: 0.8058
Test loss: 1.0413209199905396
Test accuracy: 0.8058000206947327
